In [3]:
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np
from os import listdir
import PIL

In [ ]:
# loading all of the images, resizing them, converting them 
# to grayscale, and storing them in a dictionary according to
# their respective classifications

loaded_images = {}

trick_classes = [
    '5-0 Grinds',
    '50-50 Grinds',
    'Crooks and Overcrooks',
    'Feeble Grinds',
    'Lipslides and Boardslides',
    'Nose Grinds',
    'Noseblunts and Bluntslides',
    'Noseslides and Tailslides',
    'Smith Grinds'
]


for trick in trick_classes:
    image_directory = '/Skate Images/{}'.format(trick)
    for filename in listdir(image_directory):
        if trick not in loaded_images.keys():
            loaded_images[trick] = []
        else:
            img = '/Skate Images/{}/{}'.format(trick, filename)
            loaded_images[trick].append()
            
        